### Importing Libraries

In [1]:
import pickle
import os
import pandas as pd
import numpy as np
import requests # Import the requests library
from datetime import datetime 
import pytz

### Loading the model

In [2]:
model_path = r"C:\Users\Asus\Desktop\College Projects\Seoul Bike Prediction Project\models\XGBoost_Regressor_r2_0_929_v1.pkl"
sc_dump_path = r"C:\Users\Asus\Desktop\College Projects\Seoul Bike Prediction Project\models\sc.pkl"

try:
    model = pickle.load(open(model_path, "rb"))
    SC = pickle.load(open(sc_dump_path, "rb"))
    print("Model and StandardScaler loaded successfully.")
except FileNotFoundError:
    print(f"Error: Model or StandardScaler file not found at specified paths.")
    print("Please ensure the paths are correct and the files exist.")
except Exception as e:
    print(f"An error occurred while loading the model or scaler: {e}")

Model and StandardScaler loaded successfully.


C:\Users\Asus\Anaconda3\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Define User Input Data Example

In [3]:
Date = "26/07/2023"
Hour = 5
Temperature = 25
Humidity = 97
Wind_speed = 1.11111111
Visibility = 20
Solar_Radiation = 1.0
Rainfall = 2
Snowfall = 0
Seasons = "Summer"
Holiday = "No Holiday"
Functioning_Day = "Yes"

### Helper Functions for Data Transformation

In [4]:
Holiday_dict = {"Holiday": 1, "No Holiday": 0}
Functioning_Day_dict = {"Yes": 1, "No": 0}

def get_string_to_datetime(date):
    dt = datetime.strptime(date, "%d/%m/%Y")
    return {"Day": dt.day, "Month": dt.month, "Year": dt.year, "Weekdays": dt.strftime("%A")}

def season_to_Df(selected_season):
    seasons_cols = ['Autumn', 'Spring', 'Summer', 'Winter']
    seasons_data = np.zeros((1, len(seasons_cols)), dtype="int")
    df_seasons = pd.DataFrame(seasons_data, columns=seasons_cols)
    if selected_season in seasons_cols:
        df_seasons[selected_season] = 1
    return df_seasons

def Day_to_Df(selected_weekday):
    name_days = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    name_days_data = np.zeros((1, len(name_days)), dtype="int")
    df_days = pd.DataFrame(name_days_data, columns=name_days)
    if selected_weekday in name_days:
        df_days[selected_weekday] = 1
    return df_days

### Process User Input

In [5]:
date_time_data = get_string_to_datetime(Date)

user_input_list = [
    Hour, Temperature, Humidity, Wind_speed, Visibility, Solar_Radiation, Rainfall, Snowfall,
    Holiday_dict[Holiday], Functioning_Day_dict[Functioning_Day],
    date_time_data["Day"], date_time_data["Month"], date_time_data["Year"]
]

feature_name = [
    'Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)',
    'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day',
    'Day', 'Month', 'Year'
]

df_user_input = pd.DataFrame([user_input_list], columns=feature_name)
df_seasons = season_to_Df(Seasons)
df_Weekdays = Day_to_Df(date_time_data["Weekdays"])

# Concatenate all features into a single DataFrame for prediction
df_user_pred = pd.concat([df_user_input, df_seasons, df_Weekdays], axis=1)

# Ensure the column order matches the training data features
# This is crucial for correct predictions, so explicitly define the order if possible
# (You would get the exact order from your training script)
expected_feature_order = [
    'Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)',
    'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day',
    'Day', 'Month', 'Year', 'Autumn', 'Spring', 'Summer', 'Winter', 'Friday', 'Monday',
    'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'
]

# Reindex to ensure correct column order, filling missing columns with 0 if necessary
# This handles cases where a season/weekday might not be present in a single input but was in training
df_user_pred = df_user_pred.reindex(columns=expected_feature_order, fill_value=0)


# --- 5. Scale the Input Data ---
# Corrected: Using SC instead of sc
if 'SC' in locals() and SC is not None:
    sc_user_pred = SC.transform(df_user_pred)
    # print("Scaled User Input:", sc_user_pred) # Uncomment to see the scaled input

    # --- 6. Make Prediction ---
    if 'model' in locals() and model is not None:
        prediction = model.predict(sc_user_pred).tolist()[0]
        print(f"No. of Bike Rented on {Date} at {Hour} is : {round(prediction)}")
    else:
        print("Model not loaded. Cannot make prediction.")
else:
    print("StandardScaler not loaded. Cannot scale data for prediction.")

No. of Bike Rented on 26/07/2023 at 5 is : 583


### Encapsulated Inference Function With Real Data

In [6]:
def get_current_weather_data(city_name="Seoul", api_key="YOUR_WEATHERAPI_COM_API_KEY"):
    """
    Fetches current weather data for a given city from WeatherAPI.com.
    """
    base_url = "http://api.weatherapi.com/v1/current.json"
    params = {
        "key": api_key,
        "q": city_name,
        "aqi": "no"
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        if "current" in data:
            current_data = data["current"]
            
            temperature = current_data.get("temp_c")
            humidity = current_data.get("humidity")
            wind_kph = current_data.get("wind_kph")
            wind_speed_ms = wind_kph * 0.277778 if wind_kph is not None else 0.0

            rainfall = current_data.get("precip_mm", 0)
            snowfall = 0.0 # Placeholder for snowfall

            visibility_km = current_data.get("vis_km")
            visibility_10m = (visibility_km * 1000) / 10 if visibility_km is not None else 1000

            solar_radiation = 0.0 # Placeholder for solar radiation

            return {
                "Temperature": temperature,
                "Humidity": humidity,
                "Wind_speed": wind_speed_ms,
                "Visibility": visibility_10m,
                "Solar_Radiation": solar_radiation,
                "Rainfall": rainfall,
                "Snowfall": snowfall,
            }
        else:
            print(f"Weather data: No 'current' data found in response for '{city_name}'.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during weather data processing: {e}")
        return None

# --- predict_bike_demand function (ensure this function definition is also at the top level) ---
def predict_bike_demand(
    # Ensure all these lines are indented by EXACTLY 4 spaces from 'def predict_bike_demand('
    date_str: str = datetime.now().strftime("%d/%m/%Y"),
    hour: int = datetime.now().hour,
    season: str = None,
    holiday_status: str = "No Holiday",
    functioning_day_status: str = "Yes",
    model_dir: str = r"C:\Users\Asus\Desktop\College Projects\Seoul Bike Prediction Project\models\\",
    city_for_weather: str = "Seoul",
    weatherapi_com_api_key: str = "b7d885329c5e4b149af64415252807" # Your API key
) -> int:
    model = None
    SC = None

    model_path = os.path.join(model_dir, "XGBoost_Regressor_r2_0_929_v1.pkl")
    sc_dump_path = os.path.join(model_dir, "sc.pkl")

    try:
        model = pickle.load(open(model_path, "rb"))
        SC = pickle.load(open(sc_dump_path, "rb"))
    except FileNotFoundError:
        print(f"Error: Model or StandardScaler file not found in {model_dir}.")
        print("Please ensure the path is correct and the files exist.")
        return -1
    except Exception as e:
        print(f"An error occurred while loading the model or scaler: {e}")
        return -1

    # --- Fetch Weather Data Dynamically ---
    weather_data = get_current_weather_data(city_for_weather, weatherapi_com_api_key)
    if weather_data is None:
        print("Failed to get current weather data. Cannot make prediction.")
        return -1

    Temperature = weather_data["Temperature"]
    Humidity = weather_data["Humidity"]
    Wind_speed = weather_data["Wind_speed"]
    Visibility = weather_data["Visibility"]
    Solar_Radiation = weather_data["Solar_Radiation"]
    Rainfall = weather_data["Rainfall"]
    Snowfall = weather_data["Snowfall"]

    holiday_dict = {"Holiday": 1, "No Holiday": 0}
    functioning_day_dict = {"Yes": 1, "No": 0}

    def get_string_to_datetime(date):
        dt = datetime.strptime(date, "%d/%m/%Y")
        return {"Day": dt.day, "Month": dt.month, "Year": dt.year, "Weekdays": dt.strftime("%A")}

    def get_season_from_month(month):
        if 3 <= month <= 5: return "Spring"
        elif 6 <= month <= 8: return "Summer"
        elif 9 <= month <= 11: return "Autumn"
        else: return "Winter"

    def season_to_Df(selected_season_val):
        seasons_cols = ['Autumn', 'Spring', 'Summer', 'Winter']
        seasons_data = np.zeros((1, len(seasons_cols)), dtype="int")
        df_seasons = pd.DataFrame(seasons_data, columns=seasons_cols)
        if selected_season_val in seasons_cols:
            df_seasons[selected_season_val] = 1
        return df_seasons

    def Day_to_Df(selected_weekday):
        name_days = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
        name_days_data = np.zeros((1, len(name_days)), dtype="int")
        df_days = pd.DataFrame(name_days_data, columns=name_days)
        if selected_weekday in name_days:
            df_days[selected_weekday] = 1
        return df_days

    date_time_data = get_string_to_datetime(date_str)

    if season is None:
        season = get_season_from_month(date_time_data["Month"])

    user_input_list = [
        hour, Temperature, Humidity, Wind_speed, Visibility, Solar_Radiation, Rainfall, Snowfall,
        holiday_dict.get(holiday_status, 0),
        functioning_day_dict.get(functioning_day_status, 0),
        date_time_data["Day"], date_time_data["Month"], date_time_data["Year"]
    ]

    feature_name = [
        'Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)',
        'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day',
        'Day', 'Month', 'Year'
    ]

    df_user_input = pd.DataFrame([user_input_list], columns=feature_name)
    df_seasons = season_to_Df(season)
    df_Weekdays = Day_to_Df(date_time_data["Weekdays"])

    df_user_pred = pd.concat([df_user_input, df_seasons, df_Weekdays], axis=1)

    expected_feature_order = [
        'Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)',
        'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day',
        'Day', 'Month', 'Year', 'Autumn', 'Spring', 'Summer', 'Winter', 'Friday', 'Monday',
        'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday'
    ]
    df_user_pred = df_user_pred.reindex(columns=expected_feature_order, fill_value=0)

    if SC is not None and model is not None:
        sc_user_pred = SC.transform(df_user_pred)
        prediction = model.predict(sc_user_pred).tolist()[0]
        return round(prediction)
    else:
        print("Model or StandardScaler was not loaded correctly. Cannot predict.")
        return -1

In [7]:
if __name__ == "__main__":
    # Define the Seoul time zone
    seoul_tz = pytz.timezone('Asia/Seoul')

    # Get the current time in Seoul
    seoul_current_time = datetime.now(seoul_tz)

    # Extract date and hour from Seoul's current time
    current_date_seoul = seoul_current_time.strftime("%d/%m/%Y")
    current_hour_seoul = seoul_current_time.hour # This correctly gets the hour component

    print(f"Fetching live weather for {current_date_seoul} at {current_hour_seoul}:00 (Seoul Time)...")

    predicted_bikes_live = predict_bike_demand(
        date_str=current_date_seoul, # Use Seoul's current date
        hour=current_hour_seoul,     # Use Seoul's current hour
        weatherapi_com_api_key="b7d885329c5e4b149af64415252807" # Your WeatherAPI.com key
    )

    if predicted_bikes_live != -1:
        print(f"\n--- Live Prediction for Seoul ---")
        print(f"Date: {current_date_seoul}, Hour: {current_hour_seoul}:00 (Seoul Time)")
        print(f"Predicted No. of Bikes Rented: {predicted_bikes_live}")
    else:
        print("\nCould not make a live prediction.")

Fetching live weather for 07/09/2025 at 8:00 (Seoul Time)...


C:\Users\Asus\Anaconda3\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



--- Live Prediction for Seoul ---
Date: 07/09/2025, Hour: 8:00 (Seoul Time)
Predicted No. of Bikes Rented: 460
